Ray-tracing simulation of fast camera
===

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
from raysect.optical import World
from raysect.optical.observer import PowerPipeline2D, RGBAdaptiveSampler2D, RGBPipeline2D

from cherab.phix.machine import load_pfc_mesh
from cherab.phix.observer import load_camera
from cherab.phix.plasma import load_plasma
from cherab.phix.tools.visualize import set_cbar_format

Create scene graph
---

In [ ]:
# scene world
world = World()

# import plasma
plasma, eq = load_plasma(world)

# import phix mesh
mesh = load_pfc_mesh(world, reflection=True)

# import phix camera
camera = load_camera(world)

Set up pipelines
---

In [ ]:
# RGB image and Power pipelines
# NOTE: if you excute the code in .py script, the progress image can be displayed
# when `display_progress = True`.
rgb = RGBPipeline2D(display_unsaturated_fraction=1.0, name="sRGB", display_progress=False)
power = PowerPipeline2D(display_progress=False, name="power")

# set camera's pipeline property
camera.pipelines = [rgb, power]

Set other camera parameters
---

In [ ]:
# set sampler
sampler = RGBAdaptiveSampler2D(rgb, ratio=10, fraction=0.2, min_samples=10, cutoff=0.05)

camera.frame_sampler = sampler
camera.min_wavelength = 400
camera.max_wavelength = 780
camera.spectral_rays = 1
camera.spectral_bins = 20  # spectrum resolution between 400 - 780 nm in wavelength
camera.per_pixel_samples = 10
camera.lens_samples = 10

Execute Ray-tracing
---

In [ ]:
plt.ion()
camera.observe()

## Save images and power data

In [ ]:
# save images
rgb.save("rgb.png")
power.save("power.png")

# save power array data
np.save("power.npy", power.frame.mean)

## Show the power density (irradiance) image calculated from power data

In [ ]:
# get pixel area
area = camera.pixel_area

# plot as irradiance
fig = plt.figure(dpi=150)
grids = ImageGrid(fig, 111, nrows_ncols=(1, 1), axes_pad=0.1, cbar_pad=0.0, cbar_mode="single")
mappable = grids[0].imshow(power.T / area, cmap="inferno")
grids[0].set_xlabel("width [px]")
grids[0].set_ylabel("height [px]")
cbar = plt.colorbar(mappable, cax=grids.cbar_axes[0])
set_cbar_format(cbar.ax, "eng", unit="W/m$^2$")